In [ ]:
import numpy as np

import pandas as pd

import tensorflow as tf

from keras.models import Sequential

from keras.layers import LSTM, Dense

from keras.preprocessing.sequence import TimeseriesGenerator

import plotly.graph_objs as go


In [ ]:
zephyr7b = pd.read_csv('/content/GOOG.csv')

zephyr7b.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-08-08,119.120003,120.860001,117.830002,118.139999,118.139999,17061100
1,2022-08-09,117.989998,118.199997,116.559998,117.500000,117.500000,15424300
2,2022-08-10,119.589996,121.779999,119.360001,120.650002,120.650002,20497000
3,2022-08-11,122.080002,122.339996,119.550003,119.820000,119.820000,16671600
4,2022-08-12,121.160004,122.650002,120.400002,122.650002,122.650002,16121100


In [ ]:
zephyr7b.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
245,2023-07-31,133.009995,133.830002,132.130005,133.110001,133.110001,18381900
246,2023-08-01,130.854996,132.919998,130.750000,131.889999,131.889999,22154300
247,2023-08-02,129.839996,130.419998,127.849998,128.639999,128.639999,22705800
248,2023-08-03,128.369995,129.770004,127.775002,128.770004,128.770004,15018100
249,2023-08-04,129.600006,131.929993,128.315002,128.539993,128.539993,20509500


In [ ]:
zephyr7b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       250 non-null    object 
 1   Open       250 non-null    float64
 2   High       250 non-null    float64
 3   Low        250 non-null    float64
 4   Close      250 non-null    float64
 5   Adj Close  250 non-null    float64
 6   Volume     250 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.8+ KB


In [ ]:
zephyr7b.describe()

,Open,High,Low,Close,Adj Close,Volume
count,250.000000,250.000000,250.000000,250.000000,250.000000,2.500000e+02
mean,105.696616,107.172900,104.519432,105.829780,105.829780,2.707343e+07
std,12.367202,12.436379,12.389783,12.388743,12.388743,1.111356e+07
min,85.510002,86.550003,83.449997,83.489998,83.489998,8.567800e+06
25%,95.749998,97.344997,94.440003,95.835001,95.835001,2.063240e+07
50%,102.799999,104.205002,101.857502,103.549999,103.549999,2.423745e+07
75%,117.929998,119.688748,116.782501,118.070002,118.070002,3.022488e+07
max,133.009995,134.070007,132.130005,133.110001,133.110001,9.779860e+07


In [ ]:
zephyr7b.isna().any()

Date         False
Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

In [ ]:
zephyr7b.isna().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
zephyr7b.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'], dtype='object')

In [ ]:
zephyr7b['Date'] = pd.to_datetime(zephyr7b['Date'])

zephyr7b.set_index(zephyr7b['Date'],inplace=True)

zephyr7b.head()


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2022-08-08,2022-08-08,119.120003,120.860001,117.830002,118.139999,118.139999,17061100
2022-08-09,2022-08-09,117.989998,118.199997,116.559998,117.500000,117.500000,15424300
2022-08-10,2022-08-10,119.589996,121.779999,119.360001,120.650002,120.650002,20497000
2022-08-11,2022-08-11,122.080002,122.339996,119.550003,119.820000,119.820000,16671600
2022-08-12,2022-08-12,121.160004,122.650002,120.400002,122.650002,122.650002,16121100


In [ ]:
mistral = zephyr7b.drop(['High','Low','Adj Close', 'Volume'],axis =1)

In [ ]:
mistral.head()

,Date,Open,Close
Date,,,
2022-08-08,2022-08-08,119.120003,118.139999
2022-08-09,2022-08-09,117.989998,117.500000
2022-08-10,2022-08-10,119.589996,120.650002
2022-08-11,2022-08-11,122.080002,119.820000
2022-08-12,2022-08-12,121.160004,122.650002


In [ ]:

import plotly.express as px

fig = px.line(mistral, x=mistral.index, y=['Open', 'Close'], title='Google stock prices', labels={'value': 'Price', 'Date': 'Date'}, markers=True)

fig.show()



In [ ]:
close_data = mistral['Close'].values

close_data

array([118.139999, 117.5     , 120.650002, 119.82    , 122.650002,
       122.879997, 122.510002, 120.32    , 120.860001, 118.120003,
       115.07    , 114.769997, 114.699997, 117.699997, 111.300003,
       110.339996, 109.910004, 109.150002, 110.550003, 108.68    ,
       107.480003, 110.480003, 109.419998, 111.779999, 111.870003,
       105.309998, 105.870003, 103.900002, 103.629997, 103.849998,
       101.830002, 100.010002, 100.57    ,  99.169998,  98.809998,
        98.089996, 100.739998,  98.089996,  96.150002,  99.300003,
       102.410004, 102.220001, 102.239998,  99.57    ,  98.709999,
        98.050003,  98.300003,  99.709999,  97.18    , 100.779999,
       101.389999, 100.290001, 100.529999, 101.480003, 102.970001,
       104.93    ,  94.82    ,  92.599998,  96.580002,  94.660004,
        90.5     ,  87.07    ,  83.489998,  86.699997,  88.650002,
        88.910004,  87.400002,  94.169998,  96.730003,  96.029999,
        98.720001,  98.989998,  98.5     ,  97.800003,  95.830

In [ ]:
#reshape the array into 2D-with 1 column

close_data = close_data.reshape(-1,1)

close_data

array([[118.139999],
       [117.5     ],
       [120.650002],
       [119.82    ],
       [122.650002],
       [122.879997],
       [122.510002],
       [120.32    ],
       [120.860001],
       [118.120003],
       [115.07    ],
       [114.769997],
       [114.699997],
       [117.699997],
       [111.300003],
       [110.339996],
       [109.910004],
       [109.150002],
       [110.550003],
       [108.68    ],
       [107.480003],
       [110.480003],
       [109.419998],
       [111.779999],
       [111.870003],
       [105.309998],
       [105.870003],
       [103.900002],
       [103.629997],
       [103.849998],
       [101.830002],
       [100.010002],
       [100.57    ],
       [ 99.169998],
       [ 98.809998],
       [ 98.089996],
       [100.739998],
       [ 98.089996],
       [ 96.150002],
       [ 99.300003],
       [102.410004],
       [102.220001],
       [102.239998],
       [ 99.57    ],
       [ 98.709999],
       [ 98.050003],
       [ 98.300003],
       [ 99.7

In [ ]:
split_percentage = 0.80

split = int(split_percentage * len(close_data))

split

200

In [ ]:
close_train = close_data [:split]

close_test = close_data[split:]

In [ ]:
len(close_train)

200

In [ ]:
len(close_test)

50

In [ ]:
date_train = mistral['Date'][:split]

date_test = mistral['Date'][split:]

In [ ]:
look_back = 15 #number of times look back

train_generator = TimeseriesGenerator(close_train,close_train,length=look_back,batch_size = 20)

test_generator = TimeseriesGenerator(close_test,close_test,length=look_back,batch_size = 1)


In [ ]:
model = Sequential()

model.add(LSTM(10,activation = 'relu' ,input_shape=(look_back,1)))

model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam',loss='mse')

In [ ]:
model.fit_generator(train_generator,epochs = 300,verbose =1)

Epoch 1/300


<ipython-input-74-2e6a393afadf>:1: UserWarning:

`Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.



10/10 [==============================] - 2s 7ms/step - loss: 9627.5996
Epoch 2/300
10/10 [==============================] - 0s 7ms/step - loss: 9415.1123
Epoch 3/300
10/10 [==============================] - 0s 7ms/step - loss: 9215.3428
Epoch 4/300
10/10 [==============================] - 0s 7ms/step - loss: 9003.2051
Epoch 5/300
10/10 [==============================] - 0s 7ms/step - loss: 8789.6172
Epoch 6/300
10/10 [==============================] - 0s 7ms/step - loss: 8571.0527
Epoch 7/300
10/10 [==============================] - 0s 7ms/step - loss: 8362.0020
Epoch 8/300
10/10 [==============================] - 0s 7ms/step - loss: 8155.9805
Epoch 9/300
10/10 [==============================] - 0s 7ms/step - loss: 10347.0381
Epoch 10/300
10/10 [==============================] - 0s 6ms/step - loss: 34078.4414
Epoch 11/300
10/10 [==============================] - 0s 8ms/step - loss: 7042.1616
Epoch 12/300
10/10 [==============================] - 0s 7ms/step - loss: 7743.5332
Epoch 13/30

In [ ]:
prediction = model.predict_generator(test_generator)

<ipython-input-75-342c80f366ab>:1: UserWarning:

`Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.



In [ ]:
close_train = close_train.reshape((-1))

close_test = close_test.reshape((-1))

prediction = prediction.reshape((-1))

In [ ]:
prediction

array([72.0255  , 72.46499 , 72.16354 , 71.977646, 71.1641  , 71.5361  ,
       71.430176, 70.314285, 69.7858  , 70.08287 , 69.893906, 70.08965 ,
       70.05511 , 70.63694 , 70.40979 , 70.10131 , 69.078606, 68.83305 ,
       69.209045, 70.88316 , 71.88476 , 72.16848 , 72.02996 , 71.604965,
       70.52031 , 70.23458 , 70.509384, 70.88322 , 73.0889  , 74.13067 ,
       75.55545 , 76.22964 , 76.18207 , 75.22536 , 74.83269 ],
      dtype=float32)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=close_train, mode='lines', name='Close Train'))

fig.add_trace(go.Scatter(y=close_test, mode='lines', name='Close Test'))

fig.add_trace(go.Scatter(y=prediction, mode='lines', name='Prediction'))

fig.update_layout(

    title='Google stock prediction',

    xaxis_title='Index',

    yaxis_title='Price',

    legend_title='Legend'
)

fig.show()

In [ ]:
trace1 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'data'
)

trace2 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'test'
)

trace3 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'prediction'
)

layout = go.Layout(
    title = 'Google Stock Prediction',
    xaxis = dict(title='Date'),
    yaxis = dict(title='Price')
)

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)

fig.show()